In [18]:
import numpy as np
import pandas as pd
import seaborn as sn
import os
import sklearn as sk
import statsmodels.api as sm

class country:
    def __init__(self,name,year,welfare,conso,menage,individu):
        self.country_name = name
        self.year = year
        self.welfare_data = pd.read_stata(welfare)
        self.conso_data = pd.read_stata(conso)
        self.menage_data = pd.read_stata(menage)
        self.individu_data = pd.read_stata(individu)
        
        self.merge_data = pd.merge(self.individu_data,
               self.welfare_data[['hhid','zref','dtot','hhsize']],
               on = 'hhid',
               how='outer')
        self.merge_data['constant'] =1
    
    def estimation_procedure(self,index):
        wls_model = sm.WLS(self.merge_data[index],self.merge_data['constant'], weights=self.merge_data['hhweight'])
        return wls_model.fit()
        
    def prevalence(self):
        self.merge_data['prevalence']=(self.merge_data['dtot']<
                                       (self.merge_data['zref']*self.merge_data['hhsize'])).map({False:0, True:1})
        return self.estimation_procedure('prevalence')
    
    def gap(self):
        self.merge_data['gap'] = (1-self.merge_data['dtot']/(self.merge_data['zref']*self.merge_data['hhsize']))*self.merge_data['prevalence']
        return self.estimation_procedure('gap')
    
    def severity(self):
        self.merge_data['severity'] = ((1-self.merge_data['dtot']/(self.merge_data['zref']*self.merge_data['hhsize']))**2)*self.merge_data['prevalence']
        return self.estimation_procedure('severity')
    
    def aart_welfare_index(self):
        self.merge_data['aart_welfare'] = (self.merge_data['zref']*self.merge_data['hhsize'])/self.merge_data['dtot']
        return self.estimation_procedure('aart_welfare')
    
    def aart_poverty_index(self):
        if not('aart_welfare' in self.merge_data.columns):
            self.merge_data['aart_welfare'] = (self.merge_data['zref']*self.merge_data['hhsize'])/self.merge_data['dtot']
        
        self.merge_data['aart_poverty'] = self.merge_data.apply(lambda x: max(x.aart_welfare,1),axis=1)
        return self.estimation_procedure('aart_poverty')
        

In [19]:
Benin = country(name='Benin',year='2018',welfare="Benin_2018/ehcvm_welfare_BEN2018.dta",
                conso="Benin_2018/ehcvm_conso_BEN2018.dta",
               menage="Benin_2018/ehcvm_menage_BEN2018.dta",
               individu="Benin_2018/ehcvm_individu_BEN2018.dta")

In [20]:
Benin.aart_welfare_index().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:           aart_welfare   R-squared:                      -0.000
Model:                            WLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Sat, 10 Feb 2024   Prob (F-statistic):                nan
Time:                        13:48:50   Log-Likelihood:                -43124.
No. Observations:               42343   AIC:                         8.625e+04
Df Residuals:                   42342   BIC:                         8.626e+04
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
constant       1.0133      0.003    327.342      0.000       1.007       1.019
==============================================================================
Omnibus:                    20936.386   Durbin-Watson:                   0.200
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           342572.665
Skew:                           1.983   Prob(JB):                         0.00
Kurtosis:                      16.358   Cond. No.                         1.00
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [21]:
Benin.aart_poverty_index().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:           aart_poverty   R-squared:                      -0.000
Model:                            WLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Sat, 10 Feb 2024   Prob (F-statistic):                nan
Time:                        13:48:53   Log-Likelihood:                -30691.
No. Observations:               42343   AIC:                         6.138e+04
Df Residuals:                   42342   BIC:                         6.139e+04
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
constant       1.2424      0.002    538.329      0.000       1.238       1.247
==============================================================================
Omnibus:                    37430.873   Durbin-Watson:                   0.208
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2740435.747
Skew:                           3.941   Prob(JB):                         0.00
Kurtosis:                      41.615   Cond. No.                         1.00
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
Benin.severity().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:               severity   R-squared:                       0.000
Model:                            WLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Sat, 10 Feb 2024   Prob (F-statistic):                nan
Time:                        13:15:30   Log-Likelihood:                 33087.
No. Observations:               42343   AIC:                        -6.617e+04
Df Residuals:                   42342   BIC:                        -6.616e+04
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
constant       0.0533      0.001    104.078      0.000       0.052       0.054
==============================================================================
Omnibus:                    22414.300   Durbin-Watson:                   0.200
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           142217.367
Skew:                           2.561   Prob(JB):                         0.00
Kurtosis:                      10.374   Cond. No.                         1.00
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""